In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_selection import r_regression,f_classif
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

## Loading and splitting data

In [2]:
df=pd.read_csv("../data/train.csv")
df_train=df.drop("SalePrice",axis=1)
dftest=pd.read_csv("../data/test.csv")
df_result=pd.read_csv("../data/sample_submission.csv")

In [3550]:
df.corr()["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [3551]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1]

In [3552]:
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.30,random_state=42)

## Cleaning Data Train

In [3553]:
xtrain_null= xtrain.isnull().sum()[xtrain.isnull().sum() > 0]

In [3554]:
xtrain_null.sort_values(ascending=True)

Electrical         1
MasVnrType         3
MasVnrArea         3
BsmtQual          26
BsmtCond          26
BsmtExposure      26
BsmtFinType1      26
BsmtFinType2      26
GarageCond        54
GarageQual        54
GarageFinish      54
GarageType        54
GarageYrBlt       54
LotFrontage      190
FireplaceQu      487
Fence            820
Alley            956
MiscFeature      982
PoolQC          1017
dtype: int64

 Remove the row where electrical is null

In [3555]:
i=xtrain[xtrain["Electrical"].isnull()].index.tolist()
print(i)
xtrain=xtrain.drop(i)
ytrain=ytrain.drop(i)

[1379]


In [3556]:
ytrain.shape

(1021,)

In [3557]:
xtrain_null= xtrain.isnull().sum()[xtrain.isnull().sum() > 0]

* ID column isn't usefull to predict a house price so I will remove it
* GarageYrBlt has a lot of missing value, and we have a lot of other 
<br>
  features related to the garage so I will remove it too 

In [3558]:
def delete_columns(df):
    df=df.drop(["Id","GarageYrBlt"],axis=1)
    return df

In [3559]:
xtrain=delete_columns(xtrain)

## Sort features by type

In [3560]:
categorical_features=[features for features in xtrain.columns if xtrain[features].dtype=="O"]
numerical_features=[features for features in xtrain.columns if xtrain[features].dtype!="O"]


###  Update missing categorical values

In [3561]:

xtrain[categorical_features].isnull().sum().sort_values(ascending=False)

PoolQC           1016
MiscFeature       981
Alley             955
Fence             819
FireplaceQu       486
GarageType         54
GarageCond         54
GarageQual         54
GarageFinish       54
BsmtFinType1       26
BsmtFinType2       26
BsmtQual           26
BsmtExposure       26
BsmtCond           26
MasVnrType          3
Functional          0
KitchenQual         0
Electrical          0
CentralAir          0
HeatingQC           0
Heating             0
PavedDrive          0
SaleType            0
MSZoning            0
Street              0
Condition2          0
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
BldgType            0
Foundation          0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
ExterQual           0
ExterCond           0
SaleCondition       0
dtype: int64

I chose to update with missing, because the fact that there is Null values means that its not existent for example a missing pool

In [3562]:
xtrain[numerical_features].isnull().sum().sort_values(ascending=False)

LotFrontage      190
MasVnrArea         3
WoodDeckSF         0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageCars         0
GarageArea         0
MSSubClass         0
HalfBath           0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
OpenPorchSF        0
BsmtHalfBath       0
FullBath           0
BsmtFullBath       0
GrLivArea          0
LowQualFinSF       0
2ndFlrSF           0
1stFlrSF           0
TotalBsmtSF        0
BsmtUnfSF          0
BsmtFinSF2         0
BsmtFinSF1         0
YearRemodAdd       0
YearBuilt          0
OverallCond        0
OverallQual        0
LotArea            0
YrSold             0
dtype: int64

In [3563]:
xtrain[categorical_features]=xtrain[categorical_features].fillna("Missing")
xtrain[numerical_features]=xtrain[numerical_features].fillna(0)

In [3564]:
xtrain_null= xtrain.isnull().sum()[xtrain.isnull().sum() > 0]

## Duplicated rows

In [3565]:
xtrain[xtrain.duplicated()==True]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


There is no duplicated rows

## Count all columns  unique values

In [3566]:
xtrain.nunique().sort_values(ascending=True)

C:\Users\patri\miniconda3\envs\myvenv1\lib\site-packages\IPython\core\displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


CentralAir       2
Street           2
Utilities        2
PavedDrive       3
HalfBath         3
              ... 
TotalBsmtSF    582
1stFlrSF       605
BsmtUnfSF      631
GrLivArea      679
LotArea        796
Length: 78, dtype: int64

## Count numerical columns  unique values

In [3567]:
xtrain[numerical_features]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
135,20,80.0,10400,7,6,1970,1970,288.0,0,0,...,530,98,0,0,0,0,0,0,5,2008
1452,180,35.0,3675,5,5,2005,2005,80.0,547,0,...,525,0,28,0,0,0,0,0,5,2006
762,60,72.0,8640,7,5,2009,2009,0.0,24,0,...,614,169,45,0,0,0,0,0,6,2010
932,20,84.0,11670,9,5,2006,2006,302.0,0,0,...,788,0,191,0,0,0,0,0,3,2007
435,60,43.0,10667,7,6,1996,1996,0.0,385,344,...,550,158,61,0,0,0,0,0,4,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,20,78.0,9317,6,5,2006,2006,0.0,24,0,...,440,0,22,0,0,0,0,0,3,2007
1130,50,65.0,7804,4,3,1928,1950,0.0,622,0,...,576,431,44,0,0,0,0,0,12,2009
1294,20,60.0,8172,5,7,1955,1990,0.0,167,0,...,572,0,0,0,0,0,0,0,4,2006
860,50,55.0,7642,7,8,1918,1998,0.0,0,0,...,216,0,240,0,0,0,0,0,6,2007


In [3568]:
xtrain[numerical_features].var()

MSSubClass       1.821973e+03
LotFrontage      1.281957e+03
LotArea          1.284882e+08
OverallQual      1.881308e+00
OverallCond      1.234234e+00
YearBuilt        9.452141e+02
YearRemodAdd     4.303790e+02
MasVnrArea       2.979115e+04
BsmtFinSF1       2.115855e+05
BsmtFinSF2       2.288528e+04
BsmtUnfSF        1.990807e+05
TotalBsmtSF      1.999358e+05
1stFlrSF         1.517288e+05
2ndFlrSF         1.947016e+05
LowQualFinSF     2.301382e+03
GrLivArea        2.821895e+05
BsmtFullBath     2.698181e-01
BsmtHalfBath     5.819170e-02
FullBath         2.964913e-01
HalfBath         2.441935e-01
BedroomAbvGr     6.585431e-01
KitchenAbvGr     4.502698e-02
TotRmsAbvGrd     2.660022e+00
Fireplaces       4.234872e-01
GarageCars       5.344741e-01
GarageArea       4.348537e+04
WoodDeckSF       1.735975e+04
OpenPorchSF      4.791755e+03
EnclosedPorch    3.789747e+03
3SsnPorch        1.060378e+03
ScreenPorch      3.189008e+03
PoolArea         1.575366e+03
MiscVal          1.144439e+05
MoSold    

No columns has only one value so for now we keep all of them

## Divide features by type

In [3569]:
def divide_by_type(df):  
    categorical_features=[features for features in df.columns if df[features].dtype=="O"]
    numerical_features=[features for features in df.columns if df[features].dtype!="O"]
    date_features=[features for features in df.columns if "Yr" in features or "Year" in features or "Mo" in features]
    features=[]
    for feature in numerical_features:
        if feature not in date_features:
            features.append(feature)
    numerical_features=features
    return categorical_features,numerical_features,date_features

categorical_features,numerical_features,date_features=divide_by_type(xtrain)

## Ordinal Features

In [3570]:
ordinal_numerical_features=xtrain[numerical_features].max()[xtrain[numerical_features].max() <= 14].index.tolist()
ordinal_features=[features for features in xtrain.columns if re.search('Qu$',features) or re.search('QC',features) or re.search('Qual$',features) or re.search('Cond$',features)]
ordinal_categorical_features=[features for features in ordinal_features if xtrain[features].dtype=="O"]

In [3571]:
l=[]
for feature in numerical_features:
    if feature not in (ordinal_categorical_features + ordinal_numerical_features):
        print(feature)
        l.append(feature)




s=[]
for feature in categorical_features:

    if feature not in ordinal_categorical_features + ordinal_numerical_features:
        
        s.append(feature)
        

        
        
numerical_features=l
categorical_features=s

MSSubClass
LotFrontage
LotArea
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal


In [3572]:
ordinal_categorical_features

['ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'PoolQC']

In [3573]:
numerical_features

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [3574]:
ordinal_numerical_features.remove("GarageCars")

In [3575]:
numerical_features=xtrain[numerical_features].var()[xtrain[numerical_features].var() >= 1].index.tolist()

## Outliers for future testing

In [3576]:
dict_outlier={}
for feature in numerical_features:
    # calculate summary statistics
    data_mean, data_std = xtrain[feature].mean(), xtrain[feature].std()
    # define outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    dict_outlier[feature]=xtrain[(xtrain[feature] < lower) | (xtrain[feature] > upper) ].index.tolist()


## Feature Selection

### Numerical Features 

In [3577]:
corr=r_regression(xtrain[numerical_features],ytrain)
result=np.where(corr > 0.5)[0].tolist()

In [3578]:
numerical_features

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [3579]:
selected_numerical_features=[]
for feature in numerical_features:
    print(feature)
    if numerical_features.index(feature) in result:
        selected_numerical_features.append(feature)
selected_numerical_features

MSSubClass
LotFrontage
LotArea
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal


['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']

For ordinal_numerical_features we could have used the spearman correlation because it'is based on ranks

In [3580]:
corr=r_regression(xtrain[ordinal_numerical_features],ytrain)
result1=np.where(corr > 0.5)[0].tolist()


In [3581]:
selected_numerical_ordinal_features=[]
for feature in ordinal_numerical_features:
    if ordinal_numerical_features.index(feature) in result1:
        selected_numerical_ordinal_features.append(feature)
selected_numerical_ordinal_features

['OverallQual', 'FullBath', 'TotRmsAbvGrd']

In [3582]:
corr=r_regression(xtrain[date_features],ytrain)
result2=np.where(corr > 0.5)[0].tolist()

In [3583]:
selected_date_features=[]
for feature in date_features:
    if date_features.index(feature) in result2:
        selected_date_features.append(feature)
selected_date_features

['YearBuilt', 'YearRemodAdd']

## Categorical Features

In [3584]:
dict_enc1={}
for feature in ordinal_categorical_features:
    enc = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=100)
    enc.fit(xtrain[[feature]])
    dict_enc1[feature]=enc


In [3585]:
dict_enc={}
for feature in categorical_features:
    enc = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=100)
    enc.fit(xtrain[[feature]])
    dict_enc[feature]=enc

In [3586]:
def transform_cat(df1):
    list_name=[]
    for feature,enc in dict_enc.items():
        x=enc.transform(df1[[feature]])
        df1[feature]=x
        
    return df1
cat=transform_cat(xtrain)

In [3587]:
cat["ExterCond"]

135     TA
1452    TA
762     TA
932     TA
435     TA
        ..
1095    TA
1130    TA
1294    TA
860     TA
1126    TA
Name: ExterCond, Length: 1021, dtype: object

In [3588]:
def transform_ord(df1):
    for feature,enc1 in dict_enc1.items():

        x=enc1.transform(df1[[feature]])
        df1[feature]=x
    return df1
ordinal=transform_ord(xtrain)


In [3589]:
xtrain[ordinal_categorical_features]


,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC
135,3.0,4.0,4.0,4.0,2.0,3.0,2.0,5.0,5.0,3.0
1452,3.0,4.0,2.0,4.0,2.0,3.0,3.0,5.0,5.0,3.0
762,3.0,4.0,2.0,4.0,0.0,2.0,3.0,5.0,5.0,3.0
932,0.0,4.0,0.0,1.0,0.0,0.0,2.0,5.0,5.0,3.0
435,2.0,4.0,2.0,4.0,0.0,2.0,5.0,5.0,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...
1095,2.0,4.0,2.0,4.0,2.0,2.0,2.0,5.0,5.0,3.0
1130,3.0,4.0,4.0,4.0,4.0,2.0,5.0,5.0,5.0,3.0
1294,3.0,4.0,4.0,4.0,4.0,3.0,3.0,5.0,5.0,3.0
860,2.0,4.0,4.0,4.0,2.0,2.0,2.0,5.0,5.0,3.0


In [3590]:
xtrain[categorical_features].isnull().sum()

MSZoning         0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
Foundation       0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
CentralAir       0
Electrical       0
Functional       0
GarageType       0
GarageFinish     0
PavedDrive       0
Fence            0
MiscFeature      0
SaleType         0
SaleCondition    0
dtype: int64

## Feature Engineering

In [3591]:
classif=f_classif(cat[categorical_features],ytrain)
result2=np.where(classif[1] < 0.007)[0].tolist()
select_categ=[]
for feature in categorical_features:
    if categorical_features.index(feature) in result2:
        select_categ.append(feature)
select_categ


['MSZoning',
 'Street',
 'LotShape',
 'LandSlope',
 'Neighborhood',
 'Foundation',
 'BsmtExposure',
 'CentralAir',
 'GarageType',
 'GarageFinish']

In [3592]:
classif=f_classif(xtrain[ordinal_categorical_features],ytrain)

result3=np.where(classif[1] < 0.001)[0].tolist()
select_ord=[]
for feature in ordinal_categorical_features:
    if ordinal_categorical_features.index(feature) in result3:
        select_ord.append(feature)
select_ord

['ExterQual', 'BsmtQual', 'HeatingQC', 'KitchenQual']

In [3593]:
selected_features=["Neighborhood"]+["Utilities"]+selected_date_features+selected_numerical_features+selected_numerical_ordinal_features

In [3594]:
selected_features=["Neighborhood","YearBuilt","OverallQual","GarageArea","GrLivArea","TotalBsmtSF","Utilities"]

In [3595]:
xtrain=xtrain[selected_features]

In [3596]:
scaler=MinMaxScaler()
scaler.fit(xtrain)
xtrain.loc[:,selected_features]=scaler.transform(xtrain)

C:\Users\patri\AppData\Local\Temp\ipykernel_18264\2343920160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain[selected_features]=scaler.transform(xtrain)


In [3597]:
xtrain[(xtrain["OverallQual"]==1) | (xtrain["OverallQual"]==0)].index

Int64Index([1182, 224, 523, 533, 389, 1298, 440, 1169, 185, 375, 825, 515,
            1373],
           dtype='int64')

In [3598]:
le=LinearRegression()
le.fit(xtrain,ytrain)

LinearRegression()

## Test Preprocessing

In [3599]:
xtest=delete_columns(xtest)

In [3600]:

categorical_features=[features for features in xtest.columns if xtest[features].dtype=="O"]
numerical_features=[features for features in xtest.columns if xtest[features].dtype!="O"]
xtest[categorical_features]=xtest[categorical_features].fillna("Missing")
xtest[numerical_features]=xtest[numerical_features].fillna(0)
xtest=transform_cat(xtest)
xtest=transform_ord(xtest)
xtest=xtest[selected_features]
xtest.shape
x=scaler.transform(xtest)
# xtest


In [3601]:
xtest[selected_features]=x
ypred=le.predict(xtest)

In [3602]:
le.score(xtest,ytest)

0.8005630513859212

In [3603]:
ids=dftest["Id"]
dftest=delete_columns(dftest)
categorical_features=[features for features in dftest.columns if dftest[features].dtype=="O"]
numerical_features=[features for features in dftest.columns if dftest[features].dtype!="O"]
dftest[categorical_features]=dftest[categorical_features].fillna("Missing")
dftest[numerical_features]=dftest[numerical_features].fillna(0)
dftest=transform_cat(dftest)
dftest=transform_ord(dftest)
dftest=dftest[selected_features]
dftest[selected_features]=scaler.transform(dftest)






In [3604]:
df_result["Id"]=ids
df_result["SalePrice"]=le.predict(dftest)

In [3605]:
df_result.to_csv("../data/result1.csv",index=False)

In [3606]:
dftest

,Neighborhood,YearBuilt,OverallQual,GarageArea,GrLivArea,TotalBsmtSF,Utilities
0,0.500000,0.644928,0.444444,0.514810,0.105878,0.144354,0.0
1,0.500000,0.623188,0.555556,0.220028,0.187453,0.217512,0.0
2,0.333333,0.905797,0.444444,0.339915,0.243971,0.151882,0.0
3,0.333333,0.913043,0.555556,0.331453,0.239261,0.151555,0.0
4,0.916667,0.869565,0.777778,0.356841,0.178222,0.209493,0.0
...,...,...,...,...,...,...,...
1454,0.416667,0.710145,0.333333,0.000000,0.142803,0.089362,0.0
1455,0.416667,0.710145,0.333333,0.201693,0.142803,0.089362,0.0
1456,0.458333,0.637681,0.444444,0.406206,0.167671,0.200327,0.0
1457,0.458333,0.869565,0.444444,0.000000,0.119819,0.149264,0.0


In [3607]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)
compute_rmsle(ytest,ypred)

0.24